(a) Conjunto de 1000 datos de entrenamiento y 1000 datos de test etiquetados.

In [ ]:
%matplotlib inline
import numpy as np
import random as rd
from numpy import exp, array, dot, random
from random import choice
import math as mt
import pandas as pd
import _pickle as pickle
import os
import keras

#from scipy.misc import imread
from sklearn.preprocessing import StandardScaler
from pylab import plot, ylim

from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD

from top_level_features import hog_features
from top_level_features import color_histogram_hsv
from top_level_features import extract_features

(1) Escriba una función que genere (aleatoriamente) $n$ datos etiquetados de la forma $\{(x_1,y_1),\ldots,(x_n,y_n)\}$ con $x_i \in \mathbb{R}^2$, $y_i \in \{0,1\}$, con una distribución de probabilidad que reflrja la configuración linealmente inseparable que muestra la Fig. 1. Utilice está función para generar un conjunto de 1000 datos entrenamiento y 1000 datos de pruebas. El problema de clasificación obtenido se denomina en ocasiones "XOR". ¿Porqué?

Generación de datos (training y testing sets)

In [16]:
f = open('train.txt', 'w')
g = open('test.txt', 'w')
nn = 1000

for i in range(0,nn):
    x_1 = rd.uniform(-1, 1)
    x_2 = rd.uniform(-1, 1)

    while (x_1 == 0):
        x_1 = rd.uniform(-1, 1)

    while (x_2 == 0):
        x_2 = rd.uniform(-1, 1)

    if (x_1>0):
        if(x_2>0):
            y = 1
        else: 
            y = 0
    else: 
        if(x_2<0):
            y = 1
        else: 
            y = 0
    #print>>f, x_1,x_2,y
    f.write(str(x_1)+"\t"+str(x_2)+"\t"+str(y))
    
    
for i in range(0,nn):
    x_1 = rd.uniform(-1, 1)
    x_2 = rd.uniform(-1, 1)

    while (x_1 == 0):
        x_1 = rd.uniform(-1, 1)

    while (x_2 == 0):
        x_2 = rd.uniform(-1, 1)

    if (x_1>0):
        if(x_2>0):
            y = 1
        else: 
            y = 0
    else: 
        if(x_2<0):
            y = 1
        else: 
            y = 0
    #print>>g, x_1,x_2,y
    g.write(str(x_1)+"\t"+str(x_2)+"\t"+str(y))


f.close()
g.close()
print ("ready")

ready


$\textbf{Problema de clasificación XOR:}$ El problema XOR ("o exclusiva") es uno de los  problemas más conocidos que no puede resolver el Perceptron, ya que es problema no linealmente separable (que exista una línea (o hiperplano) que separe grupos de entradas que tengan la misma salida). El perceptrón (de Rosemblatt) da lugar a una frontera lineal, ya que sólo entrega una combinación lineal (a lo más con la adición de una constante (bias)) de los inputs.

Al introducir capas al Perceptron, se logra superar la barrera de la linealidad y es posible trazar las fronteras entre ambos conjuntos (cruces y círculos).

(b) Neurona artificial individual

In [ ]:
unit_step = lambda x: 0 if x < 0.5 else 1

f = open('train.txt', 'r')
g = open('test.txt', 'r')
#nn = 1000
array_input = []
array_output = []

for line in f.readlines():
    arr = []
    for i in line.split():
        # i = float(i)
        if (i != 1 and i != 0):
            arr.append(i)
        else:
            array_output.append(int(i))
    array_input.append(arr)

training_set_inputs = np.array(array_input)
training_set_outputs = np.array(array_output).T

training_set_outputs = training_set_outputs.reshape(nn,1)

array_input = []
array_output = []

for line in g.readlines():
    arr = []
    for i in line.split():
        i = float(i)
        if (i != 1 and i != 0):
            arr.append(i)
        else:
            array_output.append(int(i))
    array_input.append(arr)


testing_set_inputs = np.array(array_input)
testing_set_outputs = np.array(array_output).T

testing_set_outputs = testing_set_outputs.reshape(nn,1)


f.close()
g.close()

rd.seed(1)

#Entrenamiento
synaptic_weights = random.random((2, 1))
h = dot(training_set_inputs, synaptic_weights)

for iteration in range(10000):
    output = 1 / (1 + np.exp(-(dot(training_set_inputs, synaptic_weights)))) # activation : Sigmoid function
    error = training_set_outputs - output # error
    sigmoid_derivative = output * (1 - output)
    adjustment = dot(training_set_inputs.T,  error* sigmoid_derivative)
    synaptic_weights += adjustment

#Evaluando con el training_set
out = 1 / (1 + np.exp(-(dot(training_set_inputs, synaptic_weights))))
diff =  training_set_outputs - out
cont = 0
for i in diff:
    j = int(mt.fabs(np.around(i, decimals=0)))
    if(j!=0):
        cont = cont + 1

print ("errores cometidos en la prediccion de training set: ",cont/10,"%")

#Evaluando con el testing_set
out = 1 / (1 + np.exp(-(dot(testing_set_inputs, synaptic_weights))))
diff =  testing_set_outputs - out
cont = 0
for i in diff:
    j = int(mt.fabs(np.around(i, decimals=0)))
    if(j!=0):
        cont = cont + 1

print ("errores cometidos en la prediccion de testing set: ",cont/10,"%")

Los resultados obtenidos reflejan que el perceptrón al trazar una frontera lineal entre ambos conjuntos, termina errando en la mitad de las clasificiones, siendo esta la recta que divide de forma diagonal el conjunto de datos (ya que es está la que crea el mejor equilibrio entre error y acierto). 

(c) Perceptron multicapa ("hecho a mano"): Preparación de los datos

In [ ]:
f = open('train.txt', 'r')
g = open('test.txt', 'r')

array_data = []

for line in f.readlines():
    arr = []
    array_data1 = []
    line = line.split(" ")
    arr.append(float(line[0]))
    arr.append(float(line[1]))
    array_data1.append(np.array(arr))
    array_data1.append(int(line[2]))
    
    array_data.append(array_data1)
    
training_data = np.array(array_data)
#print training_data

array_data = []

for line in g.readlines():
    arr = []
    array_data1 = []
    line = line.split(" ")
    arr.append(float(line[0]))
    arr.append(float(line[1]))
    array_data1.append(np.array(arr))
    array_data1.append(int(line[2]))
    
    array_data.append(array_data1)
    
testing_data = np.array(array_data)

f.close()
g.close()

Implementación del Perceptrón: Después de una serie de prubas se notó que el desempeño de este perceptrón dependía en gran medida de la inicialización de la matriz de pesos, por lo que se optó por inicializarla "a mano", obteniendo un nivel más bajo de errores.

In [ ]:
rd.seed(1)

def log_sigmoid(beta,a1,a2):
    return 1 / (1 + np.exp(-beta*dot(a1,a2)))

def ini_rand_array(A,n,m):
    if (n>0):
        A=[]
        for j in range(0,n):
            A1=[]
            for i in range(0,m):
                A1.append(random.uniform(-1,1))
            A.append(np.array(A1))
    else:
        A=[]
        for i in range(0,m):
            A.append(random.uniform(-1,1))
    return A


w_0=[]
w_1=[]

w_0 =np.array([[ 0.94067139 , 0.40326776], [-1.00636411, -0.40734721], [ 0.49253826, -0.24534223]])
w_1 = np.array([ 2.32191722 ,-2.97298485, -0.45779028])

eta = 0.001

n = 100000
beta = 15

for iteration in xrange(n):
    #Forward pass
    l_0,expected = choice(training_data)
    l_0 = np.append(l_0,1)
    l_1 = log_sigmoid(beta,l_0,w_0) 
    l_1 = np.append(l_1,1)
    l_2 = log_sigmoid(beta,l_1,w_1)
    
    #Backward pass
    #Segunda capa
    delta_1 = ( expected - l_2) * l_2*(1-l_2)
    w_1 += eta*l_1*delta_1 #Ajuste de pesos
    #Primera Capa
    delta_0 = dot(w_1,delta_1) * (l_1*(1-l_1))
    w_0 = (w_0.T + eta*l_0*delta_0).T#Ajuste de pesos

cont = 0

#Evaluando resultados con el testing_set
for l_0, exp in testing_data:
    l_0 = np.append(l_0,1)
    l_1 = np.append(log_sigmoid(beta,l_0.T,w_0),1)
    result = log_sigmoid(beta,l_1.T,w_1)

    if(unit_step(result)!=exp):
        cont+= 1

print "errores cometidos en la prediccion:", cont/10,"%"

<hr><hr><hr><hr><hr><hr><hr><hr>
## 3 (a) Contrucción de las matrices $X_{tr}$, $Y_{tr}$, $X_e$, $Y_e$, $X_{te}$ e $Y_{te}$. 

In [ ]:
def unpickle(file):
    #abre el archivo
    fo = open(file, 'rb')
    #descomprime el archivo y lo guarda en dict
    dict = pickle.load(fo)
    #cierra el archivo
    fo.close()
    #retorna dict
    return dict


#Funcion encode: codifica el output Y como vectores de dim 10, con cada dimensión representando
#una categoria del problema.
def encode(Y):
    for i in range(len(Y)):
        j=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        j[Y[i]-1]=1
        Y[i]=np.array(j)
    return np.array(Y)



#Nombre de labels
label_names = ['airplane',  #0
               'automobile',#1
               'bird',      #2
               'cat',       #3
               'deer',      #4
               'dog',       #5
               'frog',      #6
               'horse',     #7
               'ship',      #8
               'truck']     #9


def load_CIFAR_one(filename):
    #abre el archivo data_ o test_
    with open(filename, 'rb') as f:
        datadict = pickle.load(f)
        
        #matriz de 10000 x 3072 (np.array) [c/fila es una imagen]
        X1 = datadict['data']
        
        #divide X1 en 9000 de entrenamiento y 1000 para validation set.
        X2 = X1[9000:10000]  # Entrenamiento
        X1 = X1[:9000]       # Validación
        
        # lista de 10000 valores enteros en [0,9] que identifica las clases
        Y1 = datadict['labels']
        
        # divide las labels correspondientes para training y validation set.
        Y2 = Y1[9000:10000]   # Entrenamiento
        Y1 = Y1[:9000]        # Validación
        
        # Codifica ambos conjuntos de labels
        Y1 = encode(Y1)
        Y2 = encode(Y2)

        #retorna por separado data y labels
        return X1, Y1, X2 , Y2

# Funcion analoga para el conjunto de test.    
def load_CIFAR_batch(filename):
    with open(filename, 'rb') as f:
        datadict = pickle.load(f)
        Xte = datadict['data']
        Yte = datadict['labels']
        Yte = encode(Yte)
        return Xte, Yte

#Funcion que prepara las matrices pedidas juntando los 5 data_batch y extreae el validation set.    
def load_CIFAR10(PATH):
    xs = []
    ys = []
    xe = []
    ye = []
    
    for b in range(1,6):
        #abre el archivo data_batch
        f = os.path.join(PATH, 'data_batch_%d' % (b, ))
        
        # separa en X1 e Y1 data y labels del diccionario abierto para training
        # y en X2 e Y2 los de validation.
        X1,Y1,X2,Y2 = load_CIFAR_one(f)
        
        # guada c/u (y todos) en las listas xs e ys los de training y en xe e ye los de validation
        xs.append(X1)   # Training  
        ys.append(Y1)   # Training
        xe.append(X2)   # Validación
        ye.append(Y2)   # Validación
        
    # crea un array gigante con todos los datos de entrenamiento
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    
    # crea un array gigante con todos los datos de validation
    Xe  = np.concatenate(xe)
    Ye  = np.concatenate(ye) 
    del X1, Y1, X2, Y2
    
    #crea array de testing
    Xte, Yte = load_CIFAR_batch(os.path.join(PATH, 'test_batch'))
    return Xtr, Ytr, Xe, Ye, Xte, Yte


Xtr, Ytr, Xe, Ye, Xte, Yte = load_CIFAR10(".")

### (b) Funcion que escala las imágenes.

In [ ]:
def scale_img(X,Center=True,Scale=True):
    # with_mean = False disable centering
    # with_std = False disable scale
    scaler = StandardScaler(with_mean=Center,with_std=Scale).fit(X)
    X_scaled = pd.DataFrame(scaler.transform(X),X[:,0])
    return X_scaled

### (c) Redes neuronales con salida $\textit{softmax}$.

Arquitecturas probadas (número de neuronas ocultas por capa) y error de clasificación obtenido sobre el conjunto de validación:
- 10-20-10  Error: 57.76%
- 50-100-10 Error: 54.14%
- 100-500-10 Error: 55%
- 100-500-100-10 Error: 52.82% (92% de acc. sobre training set)
- 100-500-500-100-10 Error: 50.02% (100% acc. sobre training set)
- 100-100-100-100-100-10 Error: 50.16% 
- 50-100-100-100-100-10 Error: 49.92% (100% acc. sobre training set)
- 100-200-300-200-100-10 Error: 47.85% (100% acc. sobre training set)

Se probó con distintas combinaciones de funciones de activación pero las mejores siempre fueron usando exclusivamente 'relu' (al introducir 'sigmoid' inmediatamente bajaba el acc. del modelo).

Learning rate se varió entre $0.02$ y $0.001$ para las primeras 2 redes, los mejores resultados fueron con $0.009$ dejando ese valor fijo en adelante.

#### Se muestra a continuación el mejor modelo encontrado:

In [ ]:
rd.seed(1)

#Modelo
def model(Xtr,Ytr,Xe,Ye,Xte,Yte):
    model = Sequential()
    model.add(Dense(100, input_dim=Xtr.shape[1], init='uniform'))
    model.add(Activation('relu'))
    model.add(Dense(200, init='uniform'))
    model.add(Activation('relu'))
    model.add(Dense(300, init='uniform'))
    model.add(Activation('relu'))
    model.add(Dense(200, init='uniform'))
    model.add(Activation('relu'))
    model.add(Dense(100, init='uniform'))
    model.add(Activation('relu'))
    model.add(Dense(output_dim=10, init='uniform'))
    model.add(Activation('softmax'))
    sgd = SGD(lr=0.0009)
    
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    hist = model.fit(np.asmatrix(Xtr), 
                     np.asmatrix(Ytr), 
                     nb_epoch=300,
                     verbose=0, 
                     validation_data=(np.asmatrix(Xe), 
                                      np.asmatrix(Ye)
                                     )
                    )
    score = model.evaluate(np.asmatrix(Xe),np.asmatrix(Ye))
    print("Baseline Error: %.2f%%" % (100-score[1]*100))
    

    score = model.evaluate(np.asmatrix(Xte),np.asmatrix(Yte))
    print("Baseline Error Test: %.2f%%" % (100-score[1]*100))
    return hist

model(Xtr, Ytr, Xe, Ye, Xte, Yte)

Error obtenido en Validation set: 47.85%

Error obtenido sobre testing set: 47.33%


(d) Utilizando el mejor modelo obtenido en parte $(c)$ con los diferentes $extract\_features$:

In [ ]:
#extrae hog features
print 'hog_features'
model(extract_features(Xtr,[hog_features]),Ytr,extract_features(Xe,[hog_features]),Ye,extract_features(Xte,[hog_features]),Yte) 

Error obtenido en Validation set: 49.42%

Error obtenido sobre testing set: 50.17%


In [ ]:
#extrae histogramas de color
print 'color_histogram'
model(extract_features(Xtr,[color_histogram_hsv]),Ytr,extract_features(Xe,[color_histogram_hsv]),Ye,extract_features(Xte,[color_histogram_hsv]),Yte) 

Error obtenido en Validation set: 84.37%

Error obtenido sobre testing set: 83.48%


In [ ]:
#extrae todo
print 'hog_features&color_histogram'
model(extract_features(Xtr,[hog_features, color_histogram_hsv]),Ytr,extract_features(Xe,[hog_features, color_histogram_hsv]),Ye,extract_features(Xte,[hog_features, color_histogram_hsv]),Yte) 

Error obtenido en Validation set: 46.95%

Error obtenido sobre testing set: 46.59%
